In [1]:
%reload_ext tensorboard
import numpy as np
import json
import tensorflow as tf
import os

import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512)])
  except RuntimeError as e:
    print(e)
    
    
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import tf_agents
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver
import logging

import tensorflow.keras as keras

tf.compat.v1.enable_v2_behavior()
import time
import json
import datetime
import copy

from scan_gym import envs

seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

GPUs:  1


In [2]:
current_path = os.getcwd()
#sys.path.append(os.path.join(current_path, 'scan_gym/scan_gym/envs/ScannerEnv/') )
#import scanner_env
params_file = os.path.join(current_path, 'params.json') 
pm=json.load(open(params_file))

In [3]:
#env = gym.make('GatekeeperEnv-v0')
#precomputed inliers ratios for all combinations of images
envs_paths = [os.path.join(current_path, 'plants_dataset/scan_plant001/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant002/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant003/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant004/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant005/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant006/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant007/v01/inliers_ratios.npy'),
             ]
#env = suite_gym.load('ScannerEnv-v0',gym_kwargs={'setpoint':0.2,'inliers_ratios_file':os.path.join(current_path, 'scan_plant001/v01/inliers_ratios.npy'})
#tf_env = tf_py_environment.TFPyEnvironment(env)

#tf_agents.system.multiprocessing.enable_interactive_mode()#parallel environment in notebooks
#envs_callables = [lambda: suite_gym.load('ScannerEnv-v0',gym_kwargs={'setpoint':0.2,'inliers_ratios_file':x}) for x in envs_paths ]
#tf_env = tf_py_environment.TFPyEnvironment( parallel_py_environment.ParallelPyEnvironment(envs_callables) )

envs = [suite_gym.load('ScannerEnv-v0',gym_kwargs={'setpoint':0.03,'inliers_ratios_file':os.path.join(current_path,x)}) for x in envs_paths]
tf_env = tf_py_environment.TFPyEnvironment( batched_py_environment.BatchedPyEnvironment(envs,multithreading=True))#, isolation=True  )

/home/pico/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
tf_env.observation_spec()

BoundedTensorSpec(shape=(7,), dtype=tf.float32, name='observation', minimum=array([   0., -200.,    0.,    0.,   -1.,  -20.,    0.], dtype=float32), maximum=array([  1., 200.,   8.,  10.,   1.,  20.,   1.], dtype=float32))

In [5]:
#network
#scale range 0 to 1
oldmin = tf_env.observation_spec().minimum
oldmax = tf_env.observation_spec().maximum
preprocessing_layers = keras.layers.Lambda(lambda x: ((x-oldmin)*(1.- 0.)/(oldmax-oldmin)) + 0. )

fc_layer_params = tuple(pm['model']['fc_layer_params'])  #(pm['model']['fc_layer_params'],)

q_net = QNetwork(
tf_env.observation_spec(),
tf_env.action_spec(),
preprocessing_layers=preprocessing_layers,
fc_layer_params=fc_layer_params)

In [6]:
#agent
train_step = tf.Variable(0)
#optimizer = keras.optimizers.RMSprop(lr=2.5e-4, rho=0.95, momentum=0.0,
#            epsilon=0.00001, centered=True)
'''lr_decay =  keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=0.005, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.0005)
optimizer = keras.optimizers.Adam(learning_rate=lambda: lr_decay(train_step))'''


optimizer = keras.optimizers.Adam(learning_rate=pm['model']['learning_rate'])

epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=1.0, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.01) # final ε

agent = DqnAgent(tf_env.time_step_spec(),
                tf_env.action_spec(),
                q_network=q_net,
                optimizer=optimizer,
                target_update_period=pm['agent']['target_update_period'],
                td_errors_loss_fn= element_wise_squared_loss, #keras.losses.Huber(reduction="none"),
                gamma=pm['agent']['gamma'], # discount factor
                train_step_counter=train_step,
                epsilon_greedy=lambda: epsilon_fn(train_step))
agent.initialize()


In [7]:
#Replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec= agent.collect_data_spec,
    batch_size= tf_env.batch_size,
    max_length=pm['rbuffer']['max_length'])

In [8]:
#observer
#observer is just a function (or a callable object) that takes a trajectory argument,
#add_method() method (bound to the replay_buffer object) can be used as observer
replay_buffer_observer = replay_buffer.add_batch

In [9]:
#observer for training metrics
training_metrics = [
tf_metrics.NumberOfEpisodes(),
tf_metrics.AverageEpisodeLengthMetric(batch_size=len(envs)),
tf_metrics.EnvironmentSteps(),
tf_metrics.AverageReturnMetric(batch_size=len(envs)),
tf_metrics.MaxReturnMetric(batch_size=len(envs)),
tf_metrics.MinReturnMetric(batch_size=len(envs)),   
]

In [10]:
#custom observer
class ShowProgress:
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")

In [11]:
#Collect Driver
update_period = pm['collect_driver']['num_steps'] # train the model every x steps
collect_driver = DynamicStepDriver(
    tf_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + training_metrics,
    num_steps=update_period) # collect x steps for each training iteration

#+ training_metrics,

In [12]:
# random policy driver to start filling the buffer
random_collect_policy = RandomTFPolicy(tf_env.time_step_spec(),
                        tf_env.action_spec())

ns = pm['rnd_policy']['num_steps']
init_driver = DynamicStepDriver(
            tf_env,
            random_collect_policy,
            observers=[replay_buffer.add_batch],
            num_steps=ns)
#, ShowProgress(ns)],
final_time_step, final_policy_state = init_driver.run()

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [13]:
#use buffer as tf API dataset ()
dataset = replay_buffer.as_dataset(
        sample_batch_size=pm['rbuffer']['sample_batch_size'],
        num_steps=2,
        num_parallel_calls=3).prefetch(3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [14]:
log_folder = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_dir = os.path.join(current_path, log_folder)    
train_summary_writer = tf.summary.create_file_writer(
            train_dir, flush_millis=10000)
#train_summary_writer.set_as_default()

In [15]:
#convert main functions to tensorflow functions to speed up training
collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

In [16]:
#@tf.function
def train_agent(n_iterations):
    #reset metrics
    for m in training_metrics:
        m.reset()
    time_step = None
    policy_state = ()#agent.collect_policy.get_initial_state(tf_env.batch_size)
    agent.train_step_counter.assign(0)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        print("\r{} loss:{:.5f}".format(iteration, train_loss.loss.numpy()), end="")
        if iteration % 100 == 0:
            with train_summary_writer.as_default():
                tf.summary.histogram("train/hist",q_net.layers[0].weights[0],step=tf.cast(agent.train_step_counter,tf.int64))
                tf.summary.scalar('train_loss', train_loss.loss.numpy(), step=tf.cast(agent.train_step_counter,tf.int64))
                #train_summary_writer.flush()
                for train_metric in training_metrics:
                    train_metric.tf_summaries(train_step=tf.cast(agent.train_step_counter,tf.int64), step_metrics=training_metrics[:])
                train_summary_writer.flush()

In [17]:
# Launch TensorBoard with objects in the log directory
# This should launch tensorboard in your browser, but you may not see your metadata.
#%tensorboard --logdir=logs --reload_interval=15

In [18]:
#tf.summary.scalar('avgreturn', training_metrics[3].result().numpy(), step=tf.cast(agent.train_step_counter,tf.int64))

In [19]:
%time train_agent(pm['misc']['n_iterations'])

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
199999 loss:6.51390CPU times: user 2h 19min 50s, sys: 11min 53s, total: 2h 31min 43s
Wall time: 1h 38min 27s


In [20]:
replay_buffer.num_frames()

<tf.Tensor: shape=(), dtype=int64, numpy=350000>

In [21]:
env = suite_gym.load('ScannerEnv-v0',
                     gym_kwargs={'setpoint':0.03,
                     'inliers_ratios_file':os.path.join(current_path,'plants_dataset/real_plant002/v01/inliers_ratios.npy')})#envs[1]
test_env = tf_py_environment.TFPyEnvironment( env )

state = test_env.reset()
#print(state.observation[0].numpy())
last_position = env.current_position
cont = 0
time_steps = 3000
#obs = deque(maxlen=time_steps)

for i in range(time_steps):
    #obs.append(state.observation[0].numpy())
    cont +=1
    action_step = agent.policy.action(state)
    next_state = test_env.step(action_step.action)
    print(cont,':',last_position,state.observation[0].numpy(),   env.action2move(action_step.action.numpy(),state.observation[0][-1].numpy()) ) #,state.step_type.numpy()  )#,env.action2move(action_step.action,state.observation[0][-1].numpy()))#action,dir
    state = copy.deepcopy(next_state)
    last_position = env.current_position
    if state.is_last():
        print('salio',cont)
        break
        
print(env.kept_images)
print(sorted(env.kept_images))
print(env.total_reward)
print(len(env.kept_images))
print(env.total_moves)

1 : 776 [1. 0. 1. 0. 0. 0. 1.] 11118
2 : 776 [1. 0. 1. 1. 0. 0. 0.] [-13]
3 : 763 [ 0. 13.  1.  2. -1. 13.  0.] 81111
4 : 763 [ 0. 13.  1.  3. -1. 13.  1.] [4]
5 : 767 [ 0.03406594  9.          1.          4.          0.03406594 -4.
  1.        ] 11118
6 : 767 [ 0.03406594  9.          1.          5.          0.03406594 -4.
  0.        ] 88888
7 : 767 [1. 0. 1. 0. 0. 0. 0.] [-20]
8 : 747 [ 0. 20.  1.  1. -1. 20.  0.] 81111
9 : 747 [ 0. 20.  1.  2. -1. 20.  1.] [7]
10 : 754 [ 0. 13.  1.  3.  0. -7.  1.] [2]
11 : 756 [ 0. 11.  1.  4.  0. -2.  1.] [2]
12 : 758 [ 0.  9.  1.  5.  0. -2.  1.] [2]
13 : 760 [ 0.03670745  7.          1.          6.          0.03670745 -2.
  1.        ] 11118
14 : 760 [ 0.03670745  7.          1.          7.          0.03670745 -2.
  0.        ] 88888
15 : 760 [1. 0. 1. 0. 0. 0. 0.] [-20]
16 : 740 [ 0. 20.  1.  1. -1. 20.  0.] 81111
17 : 740 [ 0. 20.  1.  2. -1. 20.  1.] [7]
18 : 747 [ 0. 13.  1.  3.  0. -7.  1.] [2]
19 : 749 [ 0. 11.  1.  4.  0. -2.  1.] [2]
20

154 : 401 [1. 0. 3. 0. 0. 0. 0.] [-13]
155 : 388 [ 0.03310502 13.          3.          1.         -0.966895   13.
  0.        ] 88888
156 : 388 [1. 0. 3. 0. 0. 0. 0.] [-13]
157 : 375 [ 0.0461361 13.         3.         1.        -0.9538639 13.
  0.       ] 88888
158 : 375 [1. 0. 3. 0. 0. 0. 0.] [-13]
159 : 362 [ 0.06213365 13.          3.          1.         -0.93786633 13.
  0.        ] [-7]
160 : 355 [ 0.03530752 20.          3.          2.         -0.02682613  7.
  0.        ] 88888
161 : 355 [1. 0. 4. 0. 0. 0. 0.] [-18]
162 : 337 [ 0. 18.  4.  1. -1. 18.  0.] 81111
163 : 337 [ 0. 18.  4.  2. -1. 18.  1.] [5]
164 : 342 [ 0.05017104 13.          4.          3.          0.05017104 -5.
  1.        ] 11118
165 : 342 [ 0.05017104 13.          4.          4.          0.05017104 -5.
  0.        ] 88888
166 : 342 [1. 0. 4. 0. 0. 0. 0.] [-18]
167 : 324 [ 0.03050848 18.          4.          1.         -0.96949154 18.
  0.        ] 88888
168 : 324 [1. 0. 4. 0. 0. 0. 0.] [-18]
169 : 306 [ 0. 18.

316 : 1223 [ 0. 18.  7.  2. -1. 18.  1.] [5]
317 : 1228 [ 0. 13.  7.  3.  0. -5.  1.] [2]
318 : 1230 [ 0.03538813 11.          7.          4.          0.03538813 -2.
  1.        ] 11118
319 : 1230 [ 0.03538813 11.          7.          5.          0.03538813 -2.
  0.        ] 88888
320 : 1230 [1. 0. 7. 0. 0. 0. 0.] [-18]
321 : 1212 [ 0. 18.  7.  1. -1. 18.  0.] 81111
322 : 1212 [ 0. 18.  7.  2. -1. 18.  1.] [5]
323 : 1217 [ 0. 13.  7.  3.  0. -5.  1.] [2]
324 : 1219 [ 0.02405498 11.          7.          4.          0.02405498 -2.
  1.        ] 11118
325 : 1219 [ 0.02405498 11.          7.          5.          0.02405498 -2.
  0.        ] 88888
326 : 1219 [1. 0. 7. 0. 0. 0. 0.] [-18]
327 : 1201 [ 0. 18.  7.  1. -1. 18.  0.] 81111
328 : 1201 [ 0. 18.  7.  2. -1. 18.  1.] [5]
329 : 1206 [ 0. 13.  7.  3.  0. -5.  1.] [2]
330 : 1208 [ 0.02662037 11.          7.          4.          0.02662037 -2.
  1.        ] 11118
331 : 1208 [ 0.02662037 11.          7.          5.          0.02662037 -2.


483 : 951 [ 0. 16.  8.  2. -1. 16.  1.] [4]
484 : 955 [ 0. 12.  8.  3.  0. -4.  1.] [2]
485 : 957 [ 0. 10.  8.  4.  0. -2.  1.] [2]
486 : 959 [ 0.  8.  8.  5.  0. -2.  1.] [2]
487 : 961 [ 0.03724605  6.          8.          6.          0.03724605 -2.
  1.        ] 11118
488 : 961 [ 0.03724605  6.          8.          7.          0.03724605 -2.
  0.        ] 88888
489 : 961 [1. 0. 8. 0. 0. 0. 0.] [-16]
490 : 945 [ 0. 16.  8.  1. -1. 16.  0.] 81111
491 : 945 [ 0. 16.  8.  2. -1. 16.  1.] [4]
492 : 949 [ 0. 12.  8.  3.  0. -4.  1.] [2]
493 : 951 [ 0. 10.  8.  4.  0. -2.  1.] [2]
494 : 953 [ 0.0261067  8.         8.         5.         0.0261067 -2.
  1.       ] 11118
495 : 953 [ 0.0261067  8.         8.         6.         0.0261067 -2.
  0.       ] 88888
496 : 953 [1. 0. 8. 0. 0. 0. 0.] [-16]
497 : 937 [ 0. 16.  8.  1. -1. 16.  0.] 81111
498 : 937 [ 0. 16.  8.  2. -1. 16.  1.] [4]
499 : 941 [ 0. 12.  8.  3.  0. -4.  1.] [2]
500 : 943 [ 0. 10.  8.  4.  0. -2.  1.] [2]
501 : 945 [ 0.  8.  8.

In [22]:
policy_dir = os.path.join(current_path, 'policy')
tf_policy_saver = policy_saver.PolicySaver(agent.policy)
tf_policy_saver.save(policy_dir)

INFO:tensorflow:Assets written to: /home/pico/uni/romi/tf-agents_plant_scanner/policy/assets


INFO:tensorflow:Assets written to: /home/pico/uni/romi/tf-agents_plant_scanner/policy/assets


In [23]:
#batched_py_environment.BatchedPyEnvironment(envs=[suite.load(...) for _ in range(n)])

In [24]:
q_net.layers[0].weights[0].shape

TensorShape([7, 64])